# 資料前處理及引入函式庫

#### 1. 將作業指定的trainset 與testset併入同組pd.series，以方便計算。
#### 2.所欲預測lable需定義明確，一開始我將今日收盤價減去昨日收盤價，若大於0則得到1，是漲，相反則是跌；這麼做是想預測股票漲跌，但其實沒有意義，因為機器已經看過今日與昨日收盤價，所以預測準確率會很高(將近八成)。 因此重新定義明確的label，是"預測隔日漲跌"，將今日收盤價減去隔日收盤價，這樣預測才有意義。
#### 3.切分特徵群與標記群，並丟棄日期特徵。

In [12]:
#note: change range no influence
#note:change attributes has some influence
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, model_selection,preprocessing
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn import svm,preprocessing
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from keras import models, layers, optimizers, regularizers

 
file_name = 'stocktrain.csv'
train = pd.read_csv(file_name)

file_name2 = 'stocktest.csv'
test = pd.read_csv(file_name2)
#總共2516筆資料，為操作方便，將test併入train，並記得後252筆為TEST DATA
data=pd.concat([train,test])
#data=data.loc[0:2515,:]

value = pd.Series(data['Close Price']-data['Close Price'].shift(1),\
                  index=data.index)
value = value.bfill()
value[value>=0]=1 
value[value<0]=0 
data['Value']=value.shift(-1)
#"Value"表示隔天漲跌、漲為1

result = pd.value_counts(data['Value'])
print(result)

data=data.drop(['Date'],axis=1)
data=data.drop(0,axis=0)
#因為有與昨日價差，故第一筆資料會有null 故刪除

#data=data.drop(['Close Price'],axis=1)
#data=data.drop(['Open Price'],axis=1)
#data=data.drop(['High Price'],axis=1)
#data=data.drop(['Low Price'],axis=1)
#data=data.drop(['Volume'],axis=1)


x_data = data.loc[:, data.columns != 'Value']
x_data=preprocessing.scale(x_data)
y_data = data.loc[:,'Value']
data.head()

1.0    1372
0.0    1143
Name: Value, dtype: int64


c:\users\tomshiue\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:47: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by the scale function.


,Open Price,Close Price,High Price,Low Price,Volume,daydiffirence,dayopenchange,openchange,volumechangepercent,Value
1,929.17,927.45,936.63,919.53,5413910016,17.10,26.18,-2.63,0.252246515,1.0
2,931.17,934.70,943.85,927.28,5392620032,16.57,2,3.72,-0.003947985,0.0
3,927.45,906.65,927.45,902.37,4704940032,25.08,-3.72,-7.25,-0.146161268,1.0
4,905.73,909.73,910.00,896.81,4991549952,13.19,-21.72,-0.92,0.057419023,0.0
5,909.91,890.35,911.93,888.31,4716499968,23.62,4.18,0.18,-0.058316545,0.0


# SVM

#### 1.在SVM中，一開始只將原始資料的全部特徵丟進去訓練，但發覺速度非常慢而且全部值都猜一，可以說是沒訓練好(準確率就是1的比率，大約52%)，後來多新增了四個特徵，是關於價格差以及成交量差，大幅提高訓練速度，且也有真的在預測的行為，不是全猜一，可能是因為SVM演算法的關係，若都只有單純價格會造成演算困難。
#### 2. 採用演算模型方面，則是poly的表現最好，故以下皆是用poly討論。
#### 3.測試不同penalty，C=3時 Train 與test的平均表現較好 ，C>3發生overfitting情況。故選擇 test accuracy=54.365% 。

In [68]:
%%time

for C in range(1,6):
    print("C=%d"%C)
    L=len(x_data)-1
    train_i=int(L*(2263/2515))
    
#固定訓練集->模型固定，並預測後252天，也就是2018年的每天漲跌
    correct = 0
    train_original=train_i
    total_predict_data=L-train_i

    Data_train=x_data[0:int((2263/2515)*L)]
    value_train = y_data[0:int((2263/2515)*L)]


    classifier = svm.SVC(C, kernel='poly',gamma='auto') 
    classifier.fit(Data_train,value_train)

    acct_rate = accuracy_score(value_train, classifier.predict(Data_train))
    train_accuracy=acct_rate*100
    print("Train Accuracy = %.3f%% " %  train_accuracy)

    while train_i<L:
        
    
    
    
        Data_predict=x_data[train_i:train_i+1]
        value_real = y_data[train_i:train_i+1]
 
    
        value_predict=classifier.predict(Data_predict)
    
    #print("predictdata_index=%d  value_real=%d value_predict=%d"%(train_i,value_real,value_predict))
    
    #若預測正確，則correct計數器+1
        if(int(value_real)==int(value_predict)):
            correct=correct+1
        train_i = train_i+1

#印出準確度
    correct=correct*100/total_predict_data
    print("Test Accuracy=%.3f%%"%correct)


C=1
Train Accuracy = 55.639% 
Test Accuracy=53.968%
C=2
Train Accuracy = 55.816% 
Test Accuracy=52.778%
C=3
Train Accuracy = 55.772% 
Test Accuracy=54.365%
C=4
Train Accuracy = 55.860% 
Test Accuracy=53.175%
C=5
Train Accuracy = 55.949% 
Test Accuracy=52.381%
Wall time: 3.64 s


# Logistic Regression

#### 1.採用同樣方式前處理後的資料， test accuracy=53.571% ，略低於SVM。

In [4]:
%%time

L=len(x_data)-1
train_i=int(L*(2263/2515))


#固定訓練集->模型固定，並預測後252天，也就是2018年的每天漲跌
correct = 0
train_original=train_i
total_predict_data=L-train_i

Data_train=x_data[0:int((2263/2515)*L)]
value_train = y_data[0:int((2263/2515)*L)]


classifier = LogisticRegression(solver='lbfgs').fit(Data_train, value_train)


acct_rate = accuracy_score(value_train, classifier.predict(Data_train))
train_accuracy=acct_rate*100
print("Train Accuracy = %.3f%% " %  train_accuracy)

while train_i<L:
    
    
    
    Data_predict=x_data[train_i:train_i+1]
    value_real = y_data[train_i:train_i+1]
 
    
    value_predict=classifier.predict(Data_predict)
    
    #print("predictdata_index=%d  value_real=%d value_predict=%d"%(train_i,value_real,value_predict))
    
    #若預測正確，則correct計數器+1
    if(int(value_real)==int(value_predict)):
        correct=correct+1
    train_i = train_i+1

#印出準確度
correct=correct*100/total_predict_data
print("Test Accuracy=%.3f%%"%correct)


Train Accuracy = 54.931% 
Test Accuracy=53.571%
Wall time: 67.1 ms


# Neural Network

#### 1. 使用同一筆前處理的資料，NN模型每個預測值都猜一，所以準確度就是1的比率 ，test accuracy =52.381% 。
#### 2. 已經調整了所有可調的參數，卻仍舊無法成功訓練，應是對NN還不夠熟悉，可能是activation function或其他原因。


In [8]:
hidden_units = 1000   # how many neurons in the hidden layer
activation = 'relu'  # activation function for hidden layer
l2 = 0.0001          # regularization - how much we penalize large parameter values
learning_rate = 1e-3  # how big our steps are in gradient descent
epochs = 20          # how many epochs to train for
batch_size = 64     # how many samples to use for each gradient descent update

In [9]:
# create a sequential model
model = models.Sequential()

# add the hidden layer
model.add(layers.Dense(input_dim=9,
                       units=hidden_units, 
                       activation=activation,
                       kernel_regularizer=regularizers.l2(l2)))


model.add(layers.Dense(input_dim=hidden_units,
                       units=1, 
                       activation='softmax',
                       kernel_regularizer=regularizers.l2(l2)))




# define our loss function and optimizer
model.compile(loss='binary_crossentropy',
              # Adam is a kind of gradient descent
              optimizer=optimizers.Adam(lr=learning_rate),
              )


Instructions for updating:
Colocations handled automatically by placer.


In [10]:
class_weight = {
    0: 1.,
    1: 1144/1372
}

In [11]:
%%time 


L=len(x_data)-1
train_i=int(L*(2263/2515))


#固定訓練集->模型固定，並預測後252天，也就是2018年的每天漲跌
correct = 0
train_original=train_i
total_predict_data=L-train_i

Data_train=x_data[0:int((2263/2515)*L)]
value_train = y_data[0:int((2263/2515)*L)]


history= model.fit(Data_train, value_train, epochs=20, batch_size=batch_size,class_weight=class_weight)


acct_rate = accuracy_score(value_train, model.predict(Data_train))
train_accuracy=acct_rate*100
print("Train Accuracy = %.3f%% " %  train_accuracy)


while train_i<L:
    
    Data_train=x_data[train_i-train_original:train_i]
    value_train = y_data[train_i-train_original:train_i]
    
    Data_predict=x_data[train_i:train_i+1]
    value_real = y_data[train_i:train_i+1]
 
     
    
    value_predict=model.predict(Data_predict)
    
    #print("predictdata_index=%d  value_real=%d value_predict=%d"%(train_i,value_real,value_predict))
    
    #若預測正確，則correct計數器+1
    if(int(value_real)==int(value_predict)):
        correct=correct+1
    train_i = train_i+1

    
    

#印出準確度
correct=correct*100/total_predict_data
print("Test Accuracy=%.3f%%"%correct)



Instructions for updating:
Use tf.cast instead.
Epoch 1/20
2261/2261 [==============================] - 0s 142us/step - loss: 7.2073
Epoch 2/20
2261/2261 [==============================] - 0s 15us/step - loss: 7.2064
Epoch 3/20
2261/2261 [==============================] - 0s 16us/step - loss: 7.2062
Epoch 4/20
2261/2261 [==============================] - 0s 15us/step - loss: 7.2062
Epoch 5/20
2261/2261 [==============================] - 0s 15us/step - loss: 7.2062
Epoch 6/20
2261/2261 [==============================] - 0s 15us/step - loss: 7.2062
Epoch 7/20
2261/2261 [==============================] - 0s 16us/step - loss: 7.2062
Epoch 8/20
2261/2261 [==============================] - 0s 15us/step - loss: 7.2062
Epoch 9/20
2261/2261 [==============================] - 0s 15us/step - loss: 7.2062
Epoch 10/20
2261/2261 [==============================] - 0s 15us/step - loss: 7.2062
Epoch 11/20
2261/2261 [==============================] - 0s 15us/step - loss: 7.2062
Epoch 12/20
2261/2261 [==

predictdata_index=2428  value_real=0 value_predict=1
predictdata_index=2429  value_real=1 value_predict=1
predictdata_index=2430  value_real=0 value_predict=1
predictdata_index=2431  value_real=0 value_predict=1
predictdata_index=2432  value_real=0 value_predict=1
predictdata_index=2433  value_real=0 value_predict=1
predictdata_index=2434  value_real=1 value_predict=1
predictdata_index=2435  value_real=1 value_predict=1
predictdata_index=2436  value_real=1 value_predict=1
predictdata_index=2437  value_real=1 value_predict=1
predictdata_index=2438  value_real=1 value_predict=1
predictdata_index=2439  value_real=0 value_predict=1
predictdata_index=2440  value_real=1 value_predict=1
predictdata_index=2441  value_real=1 value_predict=1
predictdata_index=2442  value_real=1 value_predict=1
predictdata_index=2443  value_real=0 value_predict=1
predictdata_index=2444  value_real=0 value_predict=1
predictdata_index=2445  value_real=0 value_predict=1
predictdata_index=2446  value_real=0 value_pre

#  Summary
#### 從這次作業練習到了三個分類器，SVM是能較為精確且穩定的分類，LR則是較快速但粗略，NN應該是最複雜但也能最精準的，很可能沒有訓練好NN。  也從這次作業知道 ， 資料好壞的重要，可能造成garbage in garbage out，所以在訓練模型前，要好好思考所要的lable及需要的attributes是什麼，並做好適當的前處理。